In [ ]:
import wandb
from huggingface_hub.hf_api import HfFolder
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

wandb_secret = user_secrets.get_secret("wandb")
wandb.login(key=wandb_secret)

hf_secret = user_secrets.get_secret("huggingface")
HfFolder.save_token(hf_secret)

print("Login efetuado!")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: felipeverol (felipeverol-cti-renato-archer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Login efetuado!


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer 
from peft import LoraConfig, PeftModelForCausalLM
from datasets import load_dataset
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"  
)

model_name = "mistralai/Mistral-7B-Instruct-v0.3" 

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name, 
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
).to("cuda:0")

# Carrega o dataset já formatado
dataset = load_dataset("verolfelipe/drug_bank_description")


peft_config = LoraConfig(
    r=8,  # Rank dimension - typically between 4-32
    lora_alpha=16,  # LoRA scaling factor - typically 2x rank
    lora_dropout=0.05,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

# Configura o treinamento
training_args = SFTConfig(
    output_dir="./finetuned_models",
    max_steps=1000,
    per_device_train_batch_size=1,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100
)

# Treina o modelo
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    processing_class=tokenizer
)

trainer.train()

2025-05-15 18:13:21.639991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747332801.864482      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747332801.930281      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/8719 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8719 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.556600
20,1.133700
30,0.982900
40,0.927700
50,0.829300


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/peft/peft_model.py", line 1719, in forward
    return self.base_model(
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^